In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
##ngrok install
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
  | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
  | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok
import os

def load_env():
    with open('.env', 'r') as f:
        for line in f:
            if line.strip():  # 빈 줄이 아닌 경우만 처리
                key, value = line.strip().split('=')
                os.environ[key] = value

# load .env file
load_env()

ngrok_authtoken = os.getenv('NGROK_AUTH_TOKEN')

if ngrok_authtoken:
    os.system(f'ngrok config add-authtoken {ngrok_authtoken}')
else:
    print('NGROK_AUTHTOKEN not found in .env file')
!nohup ngrok http http://localhost:8000&

In [ ]:
%%capture
#FastAPI Install
!pip install fastapi
!pip install "uvicorn[standard]"

In [ ]:
%%capture
!pip install accelerate -U
!pip install -qqq -U git+https://github.com/huggingface/peft.git
!pip install -i https://pypi.org/simple/ bitsandbytes


In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import numpy as np
import bitsandbytes as bnb
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
import os
# private
def load_env():
    with open('.env', 'r') as f:
        for line in f:
            if line.strip():  # 빈 줄이 아닌 경우만 처리
                key, value = line.strip().split('=')
                os.environ[key] = value

load_env()
ACCESS_TOKEN = os.getenv('HUGGINGFACE_TOKEN')

In [ ]:
PEFT_MODEL = "/content/drive/MyDrive/Colab Notebooks/finetune_models"

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=ACCESS_TOKEN
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path, use_auth_token=ACCESS_TOKEN)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL, use_auth_token=ACCESS_TOKEN)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:711: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## inference

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 1
generation_config.top_p = 0.8
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
def tester(test_conf):
  if test_conf==1:
    generation_config.max_new_tokens = 30
    prompt = "### instruction :\nGenerate a starting word for a sentence completion game.\n\n### output :"
    output = gen(prompt).split('}')[0].split(':')[-1].strip()
    return {"message": output}

  elif test_conf==2:
    generation_config.max_new_tokens = 250
    prompt = "### instruction :\nPlease generate a word and five clues associated with it in Korean. The generated word and clues should be relevant and provide insightful hints for guessing.\n\n### output :"
    output = gen(prompt).split('}')[0].split('{')[-1].split('\n')
    return {'answer':output[1],'hints':output[3:]}

  else:
    generation_config.max_new_tokens = 12
    prompt = "### instruction :\nYour role is to assess the accuracy and consistency of the given sentence within the context of an ongoing story. Please briefly evaluate the validity of the sentence as either 'correct' or 'incorrect'.\n\n### input :\n- sentence :\nA large cave is swimming in the pond.\n\n### output :"
    output = gen(prompt).split('}')[0].split(':')[-1].strip()
    return {"message": output}

def gen(prompt):
  device = "cuda"
  encoding = tokenizer(prompt, return_tensors="pt").to(device)
  with torch.inference_mode():
    outputs = model.generate(
        input_ids = encoding.input_ids,
        attention_mask = encoding.attention_mask,
        generation_config = generation_config
    )
  return tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "")



In [ ]:
%%time
# 1 : senstart
# 2 : wmi
# 3 : correctness
test_conf=2
output = tester(test_conf)
print(output)

{'answer': '고구마', 'hints': ['길예쁘다.', '아름다운 빛을 내는 야생 꽃으로, ', '병처약으로 유명합니다.', '초식동물의 식물입니다.', '열대 지방에서 많이 발견됩니다.', '높이 100cm가 되는 큰 꽃입니다.']}
CPU times: user 30.5 s, sys: 1.65 s, total: 32.2 s
Wall time: 31.8 s


In [ ]:
# def gen(x):
#     gened = model.generate(
#         **tokenizer(
#             x,
#             return_tensors='pt',
#             return_token_type_ids=False
#         ).to('cuda'),
#         max_new_tokens=256,
#         early_stopping=True,
#         do_sample=False,
#     )
#     return tokenizer.decode(gened[0]).replace(x, "")

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!curl http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","ID":"52c9c389fa5c0a623eae06c9175eaf23","uri":"/api/tunnels/command_line","public_url":"https://ccd7-35-240-210-255.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:8000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [ ]:
        #서버 실행
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

app = FastAPI()

@app.get("/whoami") # 힌트 5개랑, 정답
def root():
    generation_config.max_new_tokens = 250
    prompt = "### instruction :\nPlease generate a word and five clues associated with it in Korean. The generated word and clues should be relevant and provide insightful hints for guessing.\n\n### output :"
    output = gen(prompt).split('}')[0].split('{')[-1].split('\n')
    return {'answer':output[1],'hints':output[3:]}


@app.get("/sentence/grading") #sentence : 판별 해주면 되는 문장
def grade(sentence):
    generation_config.max_new_tokens = 12
    prompt = f"### instruction :\nYour role is to assess the accuracy and consistency of the given sentence within the context of an ongoing story. Please briefly evaluate the validity of the sentence as either 'correct' or 'incorrect'.\n\n### input :\n- sentence :\n{sentence}\n\n### output :"
    output = gen(prompt).split('}')[0].split(':')[-1].strip()
    return {"message": output}

@app.get("/sentence/word") # 첫 시작 단어
def word():
    generation_config.max_new_tokens = 30
    prompt = "### instruction :\nGenerate a starting word for a sentence completion game.\n\n### output :"
    output = gen(prompt).split('}')[0].split(':')[-1].strip()
    return {"message": output}

import nest_asyncio
import uvicorn
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [5645]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     13.124.121.135:0 - "GET /sentence/word HTTP/1.1" 200 OK
INFO:     13.124.121.135:0 - "GET /sentence/grading?sentence=testing%2520is%2520a%2520best%2520solution HTTP/1.1" 200 OK
INFO:     13.124.121.135:0 - "GET /sentence/grading?sentence=testing%2520is%2520a%2520best%2520solution HTTP/1.1" 200 OK
INFO:     13.124.121.135:0 - "GET /sentence/word HTTP/1.1" 200 OK
INFO:     13.124.121.135:0 - "GET /sentence/word HTTP/1.1" 200 OK
INFO:     13.124.121.135:0 - "GET /sentence/grading?sentence=testing%2520is%2520a%2520best%2520solution HTTP/1.1" 200 OK
INFO:     13.124.121.135:0 - "GET /sentence/grading?sentence=testing%2520is%2520a%2520best%2520solution HTTP/1.1" 200 OK
INFO:     13.124.121.135:0 - "GET /sentence/word HTTP/1.1" 200 OK
INFO:     13.124.121.135:0 - "GET /sentence/grading?sentence=testing%2520is%2520a%2520best%2520solution HTTP/1.1" 200 OK
INFO:     13.124.121.135:0 - "GET /sentence/word HTTP/1.1" 200 OK
INFO:     13.124.121.135:0 - "GET /sentence/grading?sentence=test

In [ ]:
!ps -ef | grep ngrok